In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, first, count, lit, date_format, to_timestamp, input_file_name, regexp_replace, to_date, pandas_udf, rand
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, BooleanType, ArrayType, LongType
import fasttext
import pandas as pd
import re

In [2]:
spark = SparkSession.builder \
    .appName("LanguageDetection") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.network.timeout", "600s") \
    .getOrCreate()

initialDF = spark.read.option("recursiveFileLookup", "true").csv(
    "10_000_tweet_subset.csv", 
    header=True,  # Use the first row as column names
    inferSchema=True,  # Infer data types
    multiLine=True,  # Handle newlines within fields
    escape='\\',  # Escape character for double quotes
    quote='"',  # Define the quote character
    sep=",",  # Specify the correct delimiter
    mode="PERMISSIVE"  # Handle malformed rows gracefully
)

In [25]:
initialDF.count()

10000

In [3]:
column_values = [row["text"] for row in initialDF.select("text").collect()]

In [51]:
# column_values

In [32]:
tweet_sentences = [s.split("\n") for s in column_values]

In [4]:
spaced_tweets = [s.replace("\n", " ") for s in column_values]

In [33]:
len(tweet_sentences)

10000

In [52]:
# tweet_sentences

In [13]:
model1 = fasttext.load_model("lid.176.ftz")
model2 = fasttext.load_model("lid.176.bin")

In [29]:
model = None

def is_english(text, models):
    global model
    if model is None:
        model = fasttext.load_model(models)  # Change path
    if text is None:
        return False
    labels, _ = model.predict(text)
    return labels[0] == '__label__en'

is_english_udf = udf(is_english, BooleanType())

In [30]:
english_tweets = [tweet for tweet in spaced_tweets if is_english(tweet, "lid.176.bin")]

In [31]:
english_tweets

['# Bitcoin is becoming a nightmare for divorce lawyers http://j.mp/2o5QyOnpic.twitter.com/hPHdKOMvpv',
 'Altcoins See Red as Bitcoin Capitulation Drags Down Market https://t.co/YLetpV6ChA',
 "@Crypto_DashXRP This is where we have to be subjective to information we receive. Because they only want to cherry pick one point in time to justify their article. Look at the big picture and see if you see any correlation over the span of BTC's history with DJI and SPX. Which is almost none. https://t.co/Wp4qLnO61L",
 '5 min #RSI Signals:  $BTC - $TKS: 0.52 $BTC - $DYN: 1.17 $BTC - $OMNI: 2.44 $BTC - $BRK: 4.24 $BTC - $QWARK: 7.65 $BTC - $GRC: 7.91 $BTC - $SLR: 9.6 $BTC - $TX: 11.8 $BTC - $CRW: 19.2',
 '#BTC #BTCFX #XRP #XEM #BNB #ETH #LTC #MONA #LSK #FCT #BCH #WAVES #ADA #TRX #BTCB #BCHABC #DASH #XMR #ZEC #DCR #DGD #BTG #REP #Libra #NEO #XZC #ZEN #ETC #EOS #MCO #ATOM #QTUM #LINK #HC #GAS #GXS #WTC #GVT #OMG  こちらのURLから登録で取引手数料6ヶ月間10%割! https://t.co/vDVXxogr6W https://t.co/oRsQ0ot8WA',
 '$GXS. Pu

In [32]:
len(english_tweets)

8275

In [12]:
english_tweets

['# Bitcoin is becoming a nightmare for divorce lawyers http://j.mp/2o5QyOnpic.twitter.com/hPHdKOMvpv',
 'Altcoins See Red as Bitcoin Capitulation Drags Down Market https://t.co/YLetpV6ChA',
 "@Crypto_DashXRP This is where we have to be subjective to information we receive. Because they only want to cherry pick one point in time to justify their article. Look at the big picture and see if you see any correlation over the span of BTC's history with DJI and SPX. Which is almost none. https://t.co/Wp4qLnO61L",
 '5 min #RSI Signals:  $BTC - $TKS: 0.52 $BTC - $DYN: 1.17 $BTC - $OMNI: 2.44 $BTC - $BRK: 4.24 $BTC - $QWARK: 7.65 $BTC - $GRC: 7.91 $BTC - $SLR: 9.6 $BTC - $TX: 11.8 $BTC - $CRW: 19.2',
 '#BTC #BTCFX #XRP #XEM #BNB #ETH #LTC #MONA #LSK #FCT #BCH #WAVES #ADA #TRX #BTCB #BCHABC #DASH #XMR #ZEC #DCR #DGD #BTG #REP #Libra #NEO #XZC #ZEN #ETC #EOS #MCO #ATOM #QTUM #LINK #HC #GAS #GXS #WTC #GVT #OMG  こちらのURLから登録で取引手数料6ヶ月間10%割! https://t.co/vDVXxogr6W https://t.co/oRsQ0ot8WA',
 '$GXS. Pu

In [56]:
# Regex to extract URLs from tweet text
url_pattern = re.compile(r'https?://\S+')

# Regex pattern to identify Twitter media domains or any t.co links.
# Note: Including 't.co' means every shortened link will be flagged.
twitter_media_pattern = re.compile(r'(pic\.twitter\.com|pbs\.twimg\.com|video\.twimg\.com|t\.co)', re.IGNORECASE)

In [59]:
def tweet_contains_twitter_media(tweet):
    # Find all URLs in the tweet text.
    urls = url_pattern.findall(tweet)
    # Check each URL for a non Twitter media domain.
    for url in urls:
        if not twitter_media_pattern.search(url):
            return False
    return True

In [64]:
def has_link(tweet):
    return not url_pattern.findall(tweet)

In [65]:
# Separate tweets into two lists: those with Twitter media and those without.
tweets_with_media = []
tweets_without_media = []
tweets_no_link = []

for tweet in english_tweets:
    if tweet_contains_twitter_media(tweet):
        tweets_with_media.append(tweet)
    else:
        tweets_without_media.append(tweet)
    if has_link(tweet):
        tweets_no_link.append(tweet)

# Output results
# NOTE THAT NONE SHOULD BE REMOVED, JUST TREATED DIFFERENTLY!
print("Tweets with Twitter-handled media:", len(tweets_with_media))

print("\nTweets without Twitter-handled media:", len(tweets_without_media))

print("\nTweets without link:", len(tweets_no_link))

Tweets with Twitter-handled media: 6354

Tweets without Twitter-handled media: 1835

Tweets without link: 3222


In [62]:
tweets_with_media

['# Bitcoin is becoming a nightmare for divorce lawyers http://j.mp/2o5QyOnpic.twitter.com/hPHdKOMvpv',
 'Altcoins See Red as Bitcoin Capitulation Drags Down Market https://t.co/YLetpV6ChA',
 "@Crypto_DashXRP This is where we have to be subjective to information we receive. Because they only want to cherry pick one point in time to justify their article. Look at the big picture and see if you see any correlation over the span of BTC's history with DJI and SPX. Which is almost none. https://t.co/Wp4qLnO61L",
 '#BTC #BTCFX #XRP #XEM #BNB #ETH #LTC #MONA #LSK #FCT #BCH #WAVES #ADA #TRX #BTCB #BCHABC #DASH #XMR #ZEC #DCR #DGD #BTG #REP #Libra #NEO #XZC #ZEN #ETC #EOS #MCO #ATOM #QTUM #LINK #HC #GAS #GXS #WTC #GVT #OMG  こちらのURLから登録で取引手数料6ヶ月間10%割! https://t.co/vDVXxogr6W https://t.co/oRsQ0ot8WA',
 '$GXS. Push! Keep on rising! $BTC market on #Binance. Current Price: Ƀ 0.00019160 Sharing = Pushing!',
 '@PENTA_live https://t.co/h6nsurCNJQ #wonderland100m #Bitcoin #Ethereum #crytocurrency #MATIC

In [63]:
tweets_without_media

['19-year-old bitcoin millionaire: Heres how much you should invest in c http://zentrade.online/19-year-old-bitcoin-millionaire-heres-how-much',
 'Lightning’s Balancing Act: Challenges Face Bitcoin’s Scalability Savior: What tech challenges face the bitcoin... http://bit.ly/1RcqKVk\xa0',
 'Mr Mondrus if you are right I shall be retiring early # bitcoin https://twitter.com/cbsnews/status/936128438880043008',
 '# Fintech Industry Trends 2017 - http://buff.ly/2ulyRvV # FinancialServices # Banking # Bitcoin # wednesdaywisdom # cybersecuritypic.twitter',
 'Stratis Cryptocurrency ( $ STRAT ) Everything You Need To Know # Strat # Bitcoin # Cryptocurrencyhttp://goo.gl/dZFfCQ',
 'State Bank of India to Beta Test Blockchain Smart Contracts Next Month http://icgo.co.uk/redirect/?url=http://bitnewz.net/news/article/State',
 '# Bitcoin Price Tops $1000 in First Day of 2017 Trading http://ift.tt/2isJ83D',
 'Elections Canada Consults With Political Parties on Crypto Donations - http://bit.ly/2R9TCVs\

In [66]:
tweets_no_link

['$GXS. Push! Keep on rising! $BTC market on #Binance. Current Price: Ƀ 0.00019160 Sharing = Pushing!',
 'With #BlueSparrow, we are building the Web3 system of tomorrow that is more efficient, faster, more rewarding, and levels the playing field for everyone It’s gold of web3 #BlueSparrow #bitcoin @BlueSparrowETH  30',
 'Any cryptocurrency experts out there? Have I missed the # Bitcoin bubble?',
 'Y is it @ EmilyKDrewry # Forbes is starting to Sound more like # EntertainmentTonight ? @ Forbes Regular Misporrayal of # Bitcoin # PonziSch',
 'BTC/MONA bid:12000.00 ask:12100.00',
 '@rEtRo_sPexX @zxwife @ZX_Horace @mrbiffo @spectrum48k Manic bitcoin miner Collect special coins whilst expending huge amounts of energy',
 '# Venezuela # 30Nov Muy raro # bitcoin # BTCpic.twitter.com/2yqrbclKJg',
 '👨\u200d🚀IF BITCOINS WEEKLY MACD CONTINUES DOWNTREND, $8,000 BITCOIN IS DEFINITELY HAPPENING. THIS MARKET IS ONLY FOR SAVAGES. NO QUARTER FOR WEAK HANDS! #Blockchain #BTSE $NEXO #Dai #bitcoin #BTC #btc